# Simplify Data Loading from Spark to PyTorch using Mosaic Streaming

This notebook demonstrates the following workflow on Databricks:

1. Load data using Spark.
2. Use the `streaming.base.converters.dataframe_to_mds` API to save the dataset to a UC Volume.
3. Use `StreamingDataset` and `StreamingDataLoader` to feed the data into a single-node PyTorch model for training.
4. Use `StreamingDataset` and `StreamingDataLoader` to feed the data into a multi-node PyTorch model for training.

Requirements:
1. Databricks Runtime for ML 15.2 or higher.
2. Node configuration:
- One GPU driver.
- Two GPU workers.

## 0. Imports and Inputs

Please update the following cells with information on where data should be stored.

In [0]:
import os

NUM_EPOCHS = 10 # Update this as desired.
BATCH_SIZE = 32 # Update this as desired. Note that this is the per-device batch size.
NUM_DATA_POINTS = 500

data_storage_location = "/Volumes/mpelletier/geospatial/geotiff" # Please use a UC Volumes location so that this will work for distributed training.

if not os.path.exists(data_storage_location):
    raise Exception(f"Please create a UC Volume amd update the data_storage_location variable. The current assigned path is {data_storage_location}")

In [0]:
import io
import numpy as np
import torch
import torchvision
from PIL import Image
from functools import partial 
#from petastorm import TransformSpec
from torchvision import transforms
from streaming import StreamingDataset, StreamingDataLoader

## 1. Load data using Spark
This example uses the flowers dataset from the TensorFlow team, which contains flower photos stored under five subdirectories, one per class. The dataset is available at `/databricks-datasets/flowers`.

The example loads the flowers table, which contains the preprocessed flowers dataset, using the binary file data source. To reduce running time, this notebook uses a small subset of the flowers dataset, including ~90 training images and ~10 validation images. When you run this notebook, you can increase the number of images used for better model accuracy.

In [0]:
from pyspark.sql.functions import col

df = spark.read.table("mpelletier.geospatial.tiff") \
  .select(col("content"), col("path").alias("label"))
  
labels = df.select(col("label")).distinct().collect()
label_to_idx = {label: index for index, (label, ) in enumerate(sorted(labels))}
num_classes = len(label_to_idx)
df_train, df_val = df.randomSplit([0.7, 0.3], seed=12345)
display(df_train)

content label List(iVBORw0KGgoAAAANSUhEUgAAAU0AAAFNCAIAAAALkqqTAAEAAElEQVR4nGT9y5JlSZIkiDGL6DnXzD0is7Kqu2Z6MDRYzGKwxBdghY/Cv+Av8BVYAjsQYYAmQlNPd1V1ZlaEP8zuURVmLESPuWe3F1WGh4e52b3nqsqDmYWF/8M= (truncated), ...) dbfs:/Volumes/mpelletier/geospatial/input/data_div/sample-2_2.tif List(iVBORw0KGgoAAAANSUhEUgAAAU0AAAFOCAIAAACNBtg9AAEAAElEQVR4nFz92ZYsSZIkBhIzi4iqmbn7XWLNtau6upYG0Ohu4ODgaT57nuYDcKYf5sz0gq4tqyozIjNjuYsvZqaqIsJM8yBqNxLIeInlppubqvBGTEQiv/7Z15Y= (truncated), ...) dbfs:/Volumes/mpelletier/geospatial/input/data_div/sample-2_1.tif List(iVBORw0KGgoAAAANSUhEUgAAAU4AAAFNCAIAAADgpRGQAAEAAElEQVR4nFz9YbasuY4rBgJU5H32e/YUPIceQ89/Ol7uyhDhHwAUuzrXqluZ5+wd8X0SRYIgSBHA//f/838RFIGVJHJASCIoiaSAAQQIoBaA/1QkCEgQNKBAABA= (truncated), iVBORw0KGgoAAAANSUhEUgAAAGMAAABjCAIAAAAAWSnCAAAtsUlEQVR42nV9Z3dcybXd/AR/sKT3bBPoBggmkAA6oYmMvqFzjugAdANo5BwIAiRBgnk4HJLDnGbECZxATpC9nuzRU7L0LL0ZaUbJsr/417hynboNrVWr1+0L8AP2OmfXPvucKr7R2fIDratZx6tJ62oin830DXlogj+lS3c000V+v5m/b9bQG0cTW11NPrLIV/qjZt1J/qETL8Nl0510NRtuO/pKlh0/u+0m/iRvyI90t013kYUfyC947Dr5TfppeFqM7hb86Wkxu1vpA3vZjd6QT28r+TzoP9mGFnpgzz1tgd5DgR6yeg/j1XfY33c40H8kOHAUrdDAseDQsTcQUgIjiojO0CFvHM2NAMGXHFwCBwWOPjjoP8fvKToaQsRJYSLoOPFfjqBhEKD3FBf693OMJIhuAhP/BcPdwh48BCb2qSwdPFOwDG8rgekghUkgxdchhBFCCsPUdwQhFUAwDR4LDraHhtoZUjr7U8nq5HHkIH+Vg8YUDLEmNZpY4OhdMqDQJ/vnThsGiHzFP3LylwAgHUYNfubxwjCy0ziiSOn8QQQRgQkiZZcBhfGyk7BqNQlM5smDBgo68hU9Y9TQZ0+bSZDCC8UXiia8cEAFB3FABSlS4s9m6OBPm4CDBJEIGZ5oEiNriHHgmig6Os07nnQIKZmJTgYNy0T6zNLNzmMKpxte9L2Lw+Sxc9TsAimz284RtKsgkkWgIaHEHzBMrew9ArHnIMarl4AFso+CxZDSGFJNIvs0R5PMSofkKd0CShdH1sEpqQvGFENKc8oc5KhRgJp1p0SNoGPnbGWTaSiijCegDpKRQebBa1+MdJx6CA4cRxgX70GDPBjsqwCLLBRfvSyy/DgBcQ4GBo++0WH/AUQE/5GdB7SuA+RZBcixL1KMjMiyCfIW7C6IiXOcTb6hqScSUC6ZekpWwuVmzGWhKgqWzp4Fo2MgIC6Gt0WChcIK/RR99kiYCFKH/QwsghSPKYkIz8QmjlrTP1o+8VNHEyCpZhqSEC+Sbk38q6B2kox42TSQjLqbsxheCo4MF/QLPPskf3l4JnbLXY88CHRaCUPxH7FQamVxxFPP5MuPkTrsRzE1iJDC2fdDlnqSbpp1mYnN+wcRDC4HJewmndM5+wWabjz7hIDgMQWXiC/wVeDoIjnragbRJBJNZXrI6xwpykoco1aO0UERTRgmTFJk9ZKY6jts9mGkcOoNHvEPHqU89UPGU50HfDKIDoAoa9YAZ/tUUmf4klASxK8JMUEZSpAU/moByEYQgYQlMQJg2TQCkwY2x30W4CmhqtSA4vHVDfPuIIMJgkUCiiLFYgrxFMkyBR0JQWeT+NQBx4sUk1ubWE5raAAit5nuloYIssnfEZwlvvJoYpzlhjBZ31hVVTch7+4WxuhetiSXC6QIWJyn2lBMgbw7GhjC2x/nqU4STZ0gp1CIdTbmHd3mbDqIGp1nn6YKAhUsDgpBwZCfNhE4aMvDhMWzzHBZFBZAREHKDklKZ4LTTuW4UE806ahMB4xOYWLRhJHi2RfgRI6iKcCz7wc+ghGOKYoUQ+0AJCMIjQgiw2lXKhUllGwAMjuPEZuSaC6b5mjeD00pGtjeB/QB3fI0ylaSzpnIZM+0dvFStmrFMeUFRE6/EqRMgRQG6BCA6QjBCC+mPGn26Xwj0yUfHbBUMFRJ4ioExBR91hxA0DcQNooODYSYIZKLKkm68bls8h82KgaBiEuRBVKC0jei4uNVHnhgAUUzkZCUzDtcxxBlYPbyvKMBNUTAQkgNU6Ro3jmaFV3OBKcIIpsOuIlrd6K5ZBra5DMuYhS8tEbmkvCpAIka0MXLYy5ETQ6E7rHDvAOQkc2OQWYXW57OZQELK6YzWyGR+1lAUSJH3ISRCg6JaoYrT60TSCrBR06bElaM6SlSfAeEFsI/oCogzUEdI7LVoj9pYlqUgYfXw8p+x4tEmXp2Q8JEdkDyVWcySspxEVZk7ztI847tdwNHWd7xsCLZpyjPA1JSWfKrS9ZxOihZeD0sITBIQgm2FgylAiTiiNR0LD2JXHDtI9kVnvIoskByvIfDRAMHKHVdRQrsfQelSsBb3iGaen7G5WxZeUoRUPwNUQYigoAP1cViB0hwmVAG3dGkYmg2ZH7ZKTcBy6VZJfJmJb6kzWKXNguDpsVQg4uREcKF7YN0CZjwg+QpjhrLPhxQhxSSogw11B4aPh7ynZDZx0sTJA6wPvDtV74wvDqVoJNJ57QIK2CwwDiSYoqVMgI1A5CUSYFwgcIFVL8MBagVSEAJ4FhZ0w22PwtSTKmL1GOiPEiDiG15x0LDBCntBKj7rObJAc5BOAH5TxkuACNVfFLzwNlsVRV8B9QkOnaJjijrXMyKonSuS1ayNdCTTfcoAt3wKOzOiR8WgC2S1zlbUYOFMNQREk2Uy3Eoof0u5DtOAyqsdVAvofkf1cCqo6D+GgAIqgRRGOuS5mm51ySIDC/JUywl6U5nMPuJB5S7xQoH5W+3Qu3c5JQPjKRoHDHjhekphhF2OI8Qa4WweP+RgIgmEkfBYQYTXggpXvdxfUAQMeh+LwHirgDASHC5poh1UevJ/AIPmO8MYAzoLm4B8zgClMQxgnsfL+50j3DQWZbpiosgxQHLOA/9yooYf08bdzWPBRErEWKioUTQIRjxaAprnYSnhI8uNBTf7Fg0ATUg3gO8lCDiMdXko8/OZh9zhNUdULrDkukNmYl2w21RA3a14uNSE9osVHl2t5qyEoYWguBv7AXjaKJmOYaJ1cDBIYAUXh0olMJ6J37wAZ7iYtLG7GDpfwocQZQBYhLGOVk2XLsw4Q5EFuzHMHqyC24S/M3QoT8S3Qe3zQD6QJp25NmkcHhEDXwQfvKir1XULn5u1MmqhWx2HKZ2HEc8lEg0ddBMZNmndlyAqpSyAFZ/NksBSCET4kB6UhYlJUKG0xNKMYQR6MrIXpb0M8Gn7lbDx8uCiC2vbLqoGHEKpz2YPrzTUac8IP1fxuJBkHQooILDJ9AbrqdISaz0smT2ia6UrP4UwmpocAGtIKNJagWXIjg5nUsXWG588iuxmaD35GUpJnJNPnhZ08XgMcWCixbDPVKOU/OXBRRhcb5o6p2gAYW+BhCCTKM7YNOFmAQORS7BB+7byfYnLZtZHHVJa0EoUpB3Mt0snq/i/7p5MwbGFMao1SSdOxk+oh/F3xgs76S25D0Y6BngqoUQ+RGMkWSodooRjSn8gAJ

## 2. Save data to a UC Volume using Mosaic Streaming

The compressed training/evaluation/testing data can either be saved on the local machine for transient usage or to a UC Volume for persistent storage. When doing any form of distributed training, saving to a UC Volume is required since this data can be accessed by all of the spark workers. This data will be saved in the MDS (Mosaic Data Shard) format.

Various data types are supported, including `bytes`, `str`, `int`, `pkl`, `img`, etc. The full list can be found [here](https://docs.mosaicml.com/projects/streaming/en/stable/preparing_datasets/basic_dataset_conversion.html). Furthemore, you can specify specific compression algorithms, hashing, shard size limits, etc. View [this page](https://docs.mosaicml.com/projects/streaming/en/stable/preparing_datasets/basic_dataset_conversion.html#example-writing-a-dataset-to-mds-format) for more information.

In [0]:
import shutil
from streaming.base.converters import dataframe_to_mds

def save_mds_data(df, path, subdir):
  out_path = os.path.join(data_storage_location, subdir)
  print(f"Saving data to {out_path}")
  shutil.rmtree(out_path, ignore_errors=True)

  # This allows us to specify what the column types are for our data
  mds_kwargs = {'out': out_path, 'columns': {'content': 'bytes', 'label':'str'}}
  
  # Convert the dataset to an MDS format. It divides the dataframe into 2 parts, one parts per worker and merge the `index.json` from 2 sub-parts into one in a parent directory.
  dataframe_to_mds(df.repartition(2), merge_index=True, mds_kwargs=mds_kwargs)

save_mds_data(df_train, data_storage_location, "mds_train")
save_mds_data(df_val, data_storage_location, "mds_val")

Saving data to /Volumes/mpelletier/geospatial/geotiff/mds_train


Saving data to /Volumes/mpelletier/geospatial/geotiff/mds_val


## 3. Data Loading and Preprocessing Functions

The following functions will set up training by converting the training data to be better used by the underlying model. As you can see, the key function `get_dataloader_with_mosaic` only takes two significant lines to set up.

In [0]:
from streaming.base.util import clean_stale_shared_memory

def get_dataloader_with_mosaic(remote_path, local_path, batch_size):
  print(f"Getting data from UC Volumes at {remote_path}")

  # Utility function to clean up stale shared memory during distributed training
  clean_stale_shared_memory()

  # Creating the `StreamingDataset` object and the `StreamingDataLoader` object.
  dataset = StreamingDataset(remote=remote_path, local=local_path, shuffle=True, batch_size=batch_size)
  return StreamingDataLoader(dataset, batch_size=batch_size)

def transform_row(is_train, data_dict):
  """
  The input and output of this function must be dictionaries.
  Do data augmentation for the training dataset only.
  """
  transformers = [transforms.Lambda(lambda x: Image.open(io.BytesIO(x)))]
  if is_train:
    transformers.extend([
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
    ])
  else:
    transformers.extend([
      transforms.Resize(256),
      transforms.CenterCrop(224),
    ])
  transformers.extend([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
  ])
  
  trans = transforms.Compose(transformers)
  
  modified_data_dict = {}
  modified_data_dict['features'] = torch.stack([trans(x) for x in data_dict['content']])
  modified_data_dict['label_index'] = torch.tensor([label_to_idx[x] for x in data_dict['label']], dtype=torch.long)
  return modified_data_dict

## 4. Single Node Training Procedure

The following functions collect the `mobilenet_v2` model and train the model for `NUM_EPOCHS` epochs.

In [0]:
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from mlflow.models.signature import infer_signature


In [0]:
def get_model(lr=0.001):
  # Load a MobileNetV2 model from torchvision
  model = torchvision.models.mobilenet_v2(pretrained=True)
  # Freeze parameters in the feature extraction layers
  for param in model.parameters():
    param.requires_grad = False
    
  # Add a new classifier layer for transfer learning
  num_ftrs = model.classifier[1].in_features
  # Parameters of newly constructed modules have requires_grad=True by default
  model.classifier[1] = torch.nn.Linear(num_ftrs, num_classes)
  return model

def train_one_epoch(model, criterion, optimizer, scheduler, 
                    train_dataloader, steps_per_epoch, epoch, 
                    device, global_rank=0):
  model.train()  # Set model to training mode

  # statistics
  running_loss = 0.0
  running_corrects = 0

  # Iterate over the data for one epoch.
  for train_inputs in train_dataloader:
    train_inputs = transform_row(True, train_inputs)
    inputs, labels = train_inputs['features'].to(device), train_inputs['label_index'].to(device)
    
    # Track history in training
    with torch.set_grad_enabled(True):
      # zero the parameter gradients
      optimizer.zero_grad()

      # forward
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

      # backward + optimize
      loss.backward()
      optimizer.step()

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
  
  scheduler.step()

  epoch_loss = running_loss / (steps_per_epoch * BATCH_SIZE)
  epoch_acc = running_corrects.double() / (steps_per_epoch * BATCH_SIZE)
  if global_rank == 0:
    print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
  return epoch_loss, epoch_acc

def evaluate(model, criterion, val_dataloader, validation_steps, device, 
             metric_agg_fn=None, global_rank=0):
  model.eval()  # Set model to evaluate mode

  # statistics
  running_loss = 0.0
  running_corrects = 0

  # Iterate over all the validation data.
  for val_inputs in val_dataloader:
    val_inputs = transform_row(False, val_inputs)
    inputs, labels = val_inputs['features'].to(device), val_inputs['label_index'].to(device)

    # Do not track history in evaluation to save memory
    with torch.set_grad_enabled(False):
      # forward
      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)
      loss = criterion(outputs, labels)

    # statistics
    running_loss += loss.item()
    running_corrects += torch.sum(preds == labels.data)
  
  # Average the losses across observations for each minibatch.
  epoch_loss = running_loss / validation_steps
  epoch_acc = running_corrects.double() / (validation_steps * BATCH_SIZE)
  
  # metric_agg_fn is used in the distributed training to aggregate the metrics on all workers
  if metric_agg_fn is not None:
    epoch_loss = metric_agg_fn(epoch_loss, 'avg_loss')
    epoch_acc = metric_agg_fn(epoch_acc, 'avg_acc')
  
  if global_rank == 0:
    print('Validation Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
  return epoch_loss, epoch_acc

def train_and_evaluate(lr=0.001):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  
  model = get_model(lr=lr)
  model = model.to(device)

  criterion = torch.nn.CrossEntropyLoss()

  # Only parameters of final layer are being optimized.
  optimizer = torch.optim.SGD(model.classifier[1].parameters(), lr=lr, momentum=0.9)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
  
  # Create training Data Loader
  train_input_remote_path = "dbfs:" + os.path.join(data_storage_location, "mds_train")
  train_input_local_path = "/tmp/dataset/mds_train"
  train_dataloader = get_dataloader_with_mosaic(train_input_remote_path, train_input_local_path, BATCH_SIZE)
  train_steps_per_epoch = len(train_dataloader)

  val_input_remote_path = "dbfs:" + os.path.join(data_storage_location, "mds_val")
  val_input_local_path = "/tmp/dataset/mds_val"
  val_dataloader = get_dataloader_with_mosaic(val_input_remote_path, val_input_local_path, BATCH_SIZE)
  val_steps_per_epoch = len(val_dataloader)

  for epoch in range(NUM_EPOCHS):
    print('Epoch {}/{}'.format(epoch + 1, NUM_EPOCHS))
    print('-' * 10)

    train_loss, train_acc = train_one_epoch(model, criterion, optimizer, exp_lr_scheduler, 
                                            train_dataloader, train_steps_per_epoch, epoch, 
                                            device)
    val_loss, val_acc = evaluate(model, criterion, val_dataloader, val_steps_per_epoch, device)

  # Log the model with MLflow
  with mlflow.start_run() as run:
      # Infer the model signature
      input_example = next(iter(val_dataloader))
      input_example = transform_row(False, input_example)
      inputs, labels = input_example['features'].to(device), input_example['label_index'].to(device)
      signature = infer_signature(inputs.cpu().numpy(), model(inputs).detach().cpu().numpy())

      mlflow.pytorch.log_model(model, "model", signature=signature)
      mlflow.log_param("epochs", NUM_EPOCHS)
      mlflow.log_param("lr", lr)
      mlflow.log_metric("loss", val_loss)

      # Register the model
      model_uri = f"runs:/{run.info.run_id}/model"
      mlflow.register_model(model_uri, "mpelletier.geospatial.tiffcv")

  return val_loss
  
# Set the registry URI to Unity Catalog
mlflow.set_registry_uri("databricks-uc")

loss = train_and_evaluate()

/databricks/python/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/databricks/python/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Getting data from UC Volumes at dbfs:/Volumes/mpelletier/geospatial/geotiff/mds_train
Getting data from UC Volumes at dbfs:/Volumes/mpelletier/geospatial/geotiff/mds_val
Epoch 1/10
----------


Train Loss: 0.5805 Acc: 0.0312
Validation Loss: 2.1888 Acc: 0.0000
Epoch 2/10
----------


/databricks/python/lib/python3.11/site-packages/streaming/base/partition/orig.py:118: UserWarning: Trying to partition 1 samples over 1 canonical nodes. This will result in many samples being repeated, and depending on your batching method, batches being completely dropped. Check if your dataset has the expected number of samples, and consider decreasing the number of canonical nodes.
  warnings.warn(f'Trying to partition {num_samples} samples over {num_canonical_nodes}' +


Train Loss: 0.6032 Acc: 0.0312
Validation Loss: 2.2143 Acc: 0.0000
Epoch 3/10
----------
Train Loss: 0.5798 Acc: 0.0000
Validation Loss: 2.2636 Acc: 0.0000
Epoch 4/10
----------
Train Loss: 0.5323 Acc: 0.0000
Validation Loss: 2.3963 Acc: 0.0000
Epoch 5/10
----------
Train Loss: 0.5309 Acc: 0.0625
Validation Loss: 2.5207 Acc: 0.0000
Epoch 6/10
----------
Train Loss: 0.5140 Acc: 0.0625
Validation Loss: 2.6765 Acc: 0.0000
Epoch 7/10
----------
Train Loss: 0.4930 Acc: 0.0625
Validation Loss: 2.8274 Acc: 0.0000
Epoch 8/10
----------
Train Loss: 0.4895 Acc: 0.1250
Validation Loss: 2.8804 Acc: 0.0000
Epoch 9/10
----------
Train Loss: 0.4607 Acc: 0.0938
Validation Loss: 2.9436 Acc: 0.0000
Epoch 10/10
----------
Train Loss: 0.4797 Acc: 0.0938
Validation Loss: 3.0059 Acc: 0.0000


Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

Successfully registered model 'mpelletier.geospatial.tiffcv'.


Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

Created version '1' of model 'mpelletier.geospatial.tiffcv'.


## 5. Multi Node Distributed Training Procedure

Creating the distributed training function involves a few minor changes. The following code is an example of DDP (Distributed Data Parallel) and will use the `TorchDistributor` to start multi-node-multi-GPU training. View the [TorchDistributor docs](https://docs.databricks.com/en/machine-learning/train-model/distributed-training/spark-pytorch-distributor.html) for more details.

In [0]:
from pyspark.ml.torch.distributor import TorchDistributor
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

def distributed_train_and_evaluate(lr=0.001):
  # Loading the environment variables for distributed training
  local_rank = int(os.environ["LOCAL_RANK"])
  global_rank = int(os.environ["RANK"])
  dist.init_process_group("nccl")
  device = torch.device(local_rank)

  model = get_model(lr=lr)
  # Add the distributed data parallel module to the model
  model = model.to(device)
  model = DDP(model, device_ids=[local_rank], output_device=local_rank)

  criterion = torch.nn.CrossEntropyLoss()

  # Only parameters of final layer are being optimized.
  optimizer = torch.optim.SGD(model.module.classifier[1].parameters(), lr=lr, momentum=0.9)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
  
  # Create training Data Loader
  train_input_remote_path = "dbfs:" + os.path.join(data_storage_location, "mds_train")
  train_input_local_path = "/tmp/dataset/mds_train"
  train_dataloader = get_dataloader_with_mosaic(train_input_remote_path, train_input_local_path, BATCH_SIZE)
  train_steps_per_epoch = len(train_dataloader)

  val_input_remote_path = "dbfs:" + os.path.join(data_storage_location, "mds_val")
  val_input_local_path = "/tmp/dataset/mds_val"
  val_dataloader = get_dataloader_with_mosaic(val_input_remote_path, val_input_local_path, BATCH_SIZE)
  val_steps_per_epoch = len(val_dataloader)

  for epoch in range(NUM_EPOCHS):
    if global_rank == 0:
      print('Epoch {}/{}'.format(epoch + 1, NUM_EPOCHS))
      print('-' * 10)

    train_loss, train_acc = train_one_epoch(model, criterion, optimizer, exp_lr_scheduler, 
                                            train_dataloader, train_steps_per_epoch, epoch, 
                                            device, global_rank)
    val_loss, val_acc = evaluate(model, criterion, val_dataloader, val_steps_per_epoch, device, global_rank=global_rank)
  
  # Cleanup the distributed environment
  dist.destroy_process_group()

  return val_loss

distributor = TorchDistributor(num_processes=2, local_mode=False, use_gpu=True)
distributor.run(distributed_train_and_evaluate)